In [1]:
import os
import time

import numpy as np
from efficientnet_pytorch import EfficientNet
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from PIL import Image
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report

In [2]:
batch_size = 32
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
input_size = 224
model_path = '../models/efficientnet-b3.pth'
test_dir = '../data/test v2'
class_dict_path = '../data/training data dic.txt'

In [3]:
with open(class_dict_path, encoding='utf-8') as f:
    wordset = f.read().split('\n')

In [4]:
data_transform = transforms.Compose([
    transforms.Resize((input_size, input_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.726, 0.686, 0.695], [0.205, 0.210, 0.183])
])

image_dataset = datasets.ImageFolder(test_dir, data_transform)
dataloader = torch.utils.data.DataLoader(image_dataset,
                                         batch_size=batch_size,
                                         shuffle=True,
                                         num_workers=6) 

dataset_sizes = len(image_dataset)

class_names = image_dataset.classes

In [5]:
model_test = EfficientNet.from_pretrained('efficientnet-b3', num_classes=len(class_names))

model_test.to(device)

model_test.load_state_dict(torch.load(model_path))

model_test.eval()
print()

Loaded pretrained weights for efficientnet-b3



In [6]:
max_probs_list = []
preds_list = []
labels_list = []

with torch.no_grad():
    for inputs, labels in dataloader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model_test(inputs)
        softmax = nn.Softmax(dim=1)
        max_prods, preds = torch.max(softmax(outputs), 1)
        max_probs_list += max_prods.cpu().tolist()
        preds_list += preds.cpu().tolist()
        labels_list += labels.cpu().tolist()

In [7]:
threshold = [0, 0.3, 0.5, 0.7]

isnull_f1 = []
macro_f1 = []
weighted_f1 = []
accuracy = []

for i, t in enumerate(threshold):
    preds = np.copy(preds_list)
    labels = np.copy(labels_list)
    max_probs = np.copy(max_probs_list)
    preds[max_probs < t] = 0
    isnull_f1.append(f1_score(labels, preds, labels=[0], average="macro"))
    macro_f1.append(f1_score(labels, preds, average="macro"))
    weighted_f1.append(f1_score(labels, preds, average="weighted"))
    accuracy.append(accuracy_score(labels, preds))
    print(f'Threshold = {t}')
    print(f'isnull f1 score: {isnull_f1[i]}')
    print(f'Macro f1 score: {macro_f1[i]}')
    print(f'Weighted f1 score: {weighted_f1[i]}')
    print(f'Accuracy: {accuracy[i]}')
    print()

Threshold = 0
isnull f1 score: 0.45161290322580644
Macro f1 score: 0.8514539504441327
Weighted f1 score: 0.8693028824432998
Accuracy: 0.8529411764705882

Threshold = 0.3
isnull f1 score: 0.464
Macro f1 score: 0.8539651162790698
Weighted f1 score: 0.8718011204481794
Accuracy: 0.8550420168067226

Threshold = 0.5
isnull f1 score: 0.5230769230769231
Macro f1 score: 0.8651659125188537
Weighted f1 score: 0.8844160741219566
Accuracy: 0.865546218487395

Threshold = 0.7
isnull f1 score: 0.6433566433566433
Macro f1 score: 0.8835125073931044
Weighted f1 score: 0.912665765606942
Accuracy: 0.8907563025210085



In [8]:
print(isnull_f1 + macro_f1 + weighted_f1 + accuracy)

[0.45161290322580644, 0.464, 0.5230769230769231, 0.6433566433566433, 0.8514539504441327, 0.8539651162790698, 0.8651659125188537, 0.8835125073931044, 0.8693028824432998, 0.8718011204481794, 0.8844160741219566, 0.912665765606942, 0.8529411764705882, 0.8550420168067226, 0.865546218487395, 0.8907563025210085]
